In [1]:
import os
import re
import requests
import shutil
import xlwings as xw
import xml.etree.ElementTree as ET
import yaml
import zipfile

In [2]:
PROJECT_FOLDER = 'Frogmorton'
SHEET_NAME = 'setExcel'

CONFIGURATION_PATH = 'configuration.yaml'
IMAGES_PATH = os.path.join(PROJECT_FOLDER, 'imagesRaw')
IMAGES_ZIP_PATH = '{}/Decks/new/'.format(os.path.split(PROJECT_FOLDER)[-1])
MACROS_PATH = 'macros.xlsm'
MACROS_COPY_PATH = 'macros_copy.xlsm'
OCTGN_ZIP_PATH = 'imagesOCTGN/a21af4e8-be4b-4cda-a6b6-534f9717391f/Sets'
OUTPUT_EONS_PATH = 'imagesEons'
OUTPUT_OCTGN_PATH = ''
PROJECT_PATH = 'setGenerator.seproject'
SET_EONS_PATH = 'setEons.xml'
SET_OCTGN_PATH = 'setOCTGN'

In [3]:
# Download the cards spreadsheet from Google Drive
with open(CONFIGURATION_PATH, 'r') as f_conf:
    conf = yaml.safe_load(f_conf)

sheet_path = '{}.{}'.format(SHEET_NAME, conf['sheet_type'])
if conf['sheet_type'] == 'xlsm':
    url = 'https://drive.google.com/uc?export=download&id={}'.format(conf['sheet_gdid'])
else:
    url = 'https://docs.google.com/spreadsheets/d/{}/export?format=xlsx'.format(conf['sheet_gdid'])

myfile = requests.get(url)
with open(sheet_path, 'wb') as f_sheet:
    f_sheet.write(myfile.content)

In [5]:
# Copy Excel Sheet over to sheet that contains the macros, and run the macros
# This will create two .xml files. One in the main directory called setEons.xml, which contains the card data
# needed by Strange Eons.
# The second will be placed in the setOCTGN folder, which will be copied over to your OCTGN folder later
# in this workbook.

shutil.copyfile(MACROS_PATH, MACROS_COPY_PATH)
for folder in next(os.walk(SET_OCTGN_PATH))[1]:
    shutil.rmtree(os.path.join(SET_OCTGN_PATH, folder))

app = xw.App(visible=False)  # IF YOU WANT EXCEL TO RUN IN BACKGROUND
xlws = {}

xlwb1 = xw.Book(sheet_path)
try:
    xlws['ws1a'] = xlwb1.sheets['Sets']
    xlws['ws1b'] = xlwb1.sheets['Card Data']

    xlwb2 = xw.Book(MACROS_COPY_PATH)
    try:
        xlws['ws2a'] = xlwb2.sheets['Sets']
        xlws['ws2b'] = xlwb2.sheets['Card Data']

        newVal = xlws['ws1a'].range('A3:B3').value  # get set id/name
        xlws['ws2a'].range('A3:B3').value = newVal  # update set id/name

        newVal = xlws['ws1b'].range('A2:AV1000').value  # get card data
        xlws['ws2b'].range('A2:AV1000').value = newVal  # update card data

        # This macro will generate a folder under setOCTGN, which which will be copied over
        # to OCTGN\GameDatabase\a21af4e8-be4b-4cda-a6b6-534f9717391f\Sets 
        macro1 = xlwb2.macro('SaveOCTGN')
        macro1()

        # This macro will generate a file called setEons.xml, which is used by the Strange Eons scripts.
        macro2 = xlwb2.macro('SaveXML')
        macro2()

        xlwb2.save()
    finally:
        xlwb2.close()
finally:
    xlwb1.close()

In [6]:
# Delete old image files
for _, _, filenames in os.walk(IMAGES_PATH):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            os.remove(os.path.join(IMAGES_PATH, filename))

    break

# Parse artwork images
images = {}
for _, _, filenames in os.walk(conf['artwork_path']):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            card_number = int(filename.split('_')[0])
            images[card_number] = filename

    break

# Copy artwork images and update generated XML with filenames
tree = ET.parse(SET_EONS_PATH)
root = tree.getroot()

for card in root[0]:
    card_number = int([p for p in card
                       if p.attrib['name'] == 'Card Number'
                      ][0].attrib['value'])
    filename = images.get(card_number)
    if not filename:
        continue

    shutil.copyfile(os.path.join(conf['artwork_path'], filename),
                    os.path.join(IMAGES_PATH, filename))
    prop = ET.SubElement(card, 'property')
    prop.set('name', 'Artwork')
    prop.set('value', filename)

    is_artist = [p for p in card if p.attrib['name'] == 'Artist']
    if not is_artist and '_Artist_' in filename:
        artist = '.'.join('_Artist_'.join(
            filename.split('_Artist_')[1:]).split('.')[:-1])
        artist = artist.replace('_', ' ')
        prop = ET.SubElement(card, 'property')
        prop.set('name', 'Artist')
        prop.set('value', artist)

tree.write(SET_EONS_PATH)

In [7]:
# Create Strange Eons project
with zipfile.ZipFile(PROJECT_PATH, 'w') as zip_obj:
    for root, _, filenames in os.walk(PROJECT_FOLDER):
        for filename in filenames:
            if filename != '.gitignore':
                zip_obj.write(os.path.join(root, filename))

### Strange Eons part:

1. Open the `setGenerator.seproject` generated above.
2. Run the `makeCards` script by double clicking it (see `Script` folder).  This will take the templates, fill them with the values from the `setEons.xml` generated above, and save them under the `Decks/new` folder in Strange Eons.
3. Right-click the `Decks/new` folder, click on "Bulk Export", choose the settings you want, and hit "Export".
4. Once complete, close Strange Eons.

In [8]:
# Delete old image files
for _, _, filenames in os.walk(OUTPUT_EONS_PATH):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            os.remove(os.path.join(OUTPUT_EONS_PATH, filename))

    break

# Copy generated images from the project zip
with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist() if f.startswith(IMAGES_ZIP_PATH)
                and f.split('.')[-1] in ('jpg', 'png')]
    for filename in filelist:
        with zip_obj.open(filename) as zip_file:
            with open(os.path.join(OUTPUT_EONS_PATH,
                      os.path.split(filename)[-1]), 'wb') as output_file:
                shutil.copyfileobj(zip_file, output_file)

In [10]:
# Make an image pack

# OCTGN Set ID obtained by looking at the folders generated by the excel SaveOCTGN script
octgnid = next(os.walk(SET_OCTGN_PATH))[1][0]

pack_path = os.path.join(OUTPUT_OCTGN_PATH, 'imagePack_{}.o8c'.format(octgnid))
with zipfile.ZipFile(pack_path, 'w', zipfile.ZIP_DEFLATED) as zip_obj:
    for _, _, filenames in os.walk(OUTPUT_EONS_PATH):
        for filename in filenames:
            if filename.split('.')[-1] not in ('jpg', 'png'):
                continue

            oldfile = os.path.join(OUTPUT_EONS_PATH, filename)
            parts = filename.split('.')
            parts[0] = re.sub('-1$', '', re.sub('-2$', '.B', parts[0]))
            filename = '.'.join(parts)
            if len(filename) > 50: 
                filename = filename[50:]

            newfile = '{}/{}/Cards/{}'.format(OCTGN_ZIP_PATH, octgnid, filename)
            zip_obj.write(oldfile, newfile)

        break

# Now there should be an imagePack_<octgnid>.o8c file.
# Open that using the "Add Image Packs" button from within OCTGN to add it.

In [ ]:
# Copy set folder to OCTGN directory
from distutils.dir_util import copy_tree

dirOCTGN = os.path.expanduser('~')+'\\Documents\\OCTGN\\'
if not os.path.isdir(dirOCTGN): print('Invalid path!')

setspath = dirOCTGN+'\\GameDatabase\\a21af4e8-be4b-4cda-a6b6-534f9717391f\\Sets\\'

if os.path.isdir(dirOCTGN):
    if os.path.isdir('.\\setOCTGN\\'+octgnid):
        copy_tree('.\\setOCTGN\\'+octgnid, setspath+octgnid)
        print('Copied successfully.')
    else:
        print('No folder found to move.')
else:
    print('OCTGN Sets folder not found.')